In [ ]:
# Importeer benodigde modules
import sys
import warnings
import textwrap
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt

# Make Confusion Matrix
Dit script is bedoelt om Confusion matrixen te maken van validatie data uit het script "HerhaaldelijkeValidatie.ipynb". Het script bied de mogelijkheid om geclusterde matrixen te maken van clusters die niet bij de "HerhaaldelijkeValidatie.ipynb" zijn gebruikt. Daarnaast is het mogelijk om namen van labels en variabel te corrigeren waar nodig.

**LET OP!** Wanneer de opgeslagen matrixen toegevoegd worden aan een Microsoft word document (en mogelijk anderen bestandstype) kan de kwaliteit van de afbeelding negatief beïnvloed worden om dit tegen te gaan kun je [deze instelling](https://support.microsoft.com/en-us/office/change-the-default-resolution-for-inserting-pictures-in-office-f4aca5b4-6332-48c6-9488-bf5e0094a7d2) of [deze instelling](https://support.microsoft.com/en-us/office/turn-off-picture-compression-81a6b603-0266-4451-b08e-fc1bf58da658) aanpassen.

---
<blockquote style="border-left: 5px solid red; padding-left: 10px;">

## **Attentie!** 
<code> Update onderstaande cell voordat je de het script runt </code></blockquote>
 

In [ ]:
# Input excel die gegenereerd is met het script "Herhaaldelijke Validatie"
InputExcelLocation : Path = Path(r"pad/naar/jouw/excel/bestand.xlsx")
# Bijvoorbeeld: Path(r"pad/naar/jouw/excel/bestand.xlsx")

# Output map waar de nieuwe confusion matrixen worden opgeslagen.
OutputFolder : Path = Path("Nieuwe Map")
# Bijvoorbeeld: Path(r"pad/naar/jouw/output/folder") OF Path("Nieuwe Map")

# Welke variabele wil je combineren? Bijv. "0002_A52_flits_1_0_0" variabele 2= A52 (Rest van dataset: ip7 of ip13), variabele 3= flits (Rest van dataset: Omgeving)
    # dus met Clustering = [2, 3] Worden er Matrixen gemaakt voor alle combinaties van variabele [A52, ip7, ip13] met [Flits, Omgeving]
Clustering : list[int] = []
# Bijvoorbeeld: [0, 2] OF []

# Wanneer TitleOverwrite = variabel i word de titel van de grafiek alleen de waarde van de opgeven variabel.
    # anders TitleOverwrite = None
TitleOverwrite : int = None
# Bijvoorbeeld: 1 OF 5 OF None

# Voor variabelen met verschillende namen maar die wel als één geclusterd moeten worden. Bijv. "0002_A52_flits_1_0_0" variabele 2= A52 (Rest van dataset: ip7 of ip13)
    # VariabelenMatch = {"Iphone" : ["ip7", "ip13"]} zocht er voor dat wanneer een variabele ip7 of ip13 is deze in de clustering als "Iphone" herkent worden.
    # In de clusters komt dus allen A52 en Iphone terug voor variabele 2
VariabelenMatch : dict[str, list[str]] = {} # HOOFDLETTER GEVOELIG!
# Bijvoorbeeld: {"Ip" : ["ip7", "ip13"], "VariabelNaam": ["OGVariabel1", "OGVariabel2", "OGVariabel5"]}

# Welke van de boven gegeven variabelen wil je niet terug zien in je output? Bijv. Blacklist : list[str] = ["A52"] met de clusters uit bovenstaande comment ^^^
    # zorgt ervoor dat clusters "A52 - Flits" en "A52 - Omgeving" niet in de output terug te vinden zijn
    # de Blacklist word zowel voor als na de VariabelenMatch toegepast
Blacklist : list[str] = []
# Bijvoorbeeld: ["A52", "Ip"] OF []

# Geef hier aan wanneer de spelling van labels moet veranderen {Oud: Nieuw}
LabelMatch : dict[str: str] = {
    "Oud": "Nieuw",
} # HOOFDLETTER GEVOELIG!
# Bijvoorbeeld: {"ip7": "IP7", "ip13": "IP13"} OF {"Oud": "Nieuw"} OF {}

# Bepaal handmatig de schaal verdeling van de standaard confusion matrix. Wanneer leeg word 0 tot (2 * mediaan matrix) gebruikt
Schaal : list[int] = [0, 200] #[min, max]
# Bijvoorbeeld: [0, 200] OF [20, 50] OF []

---
## **Code**

In [ ]:
# Variabelen die de opmaak van de matrix bepalen gecentreerd voor makkelijk updaten
FontSize : float = 1.2
LineWidth : float =1, 
LineColor : str = 'whitesmoke'
FaceColor : str = 'white'
Dpi : int = 220
MaxCharacterLength : int = 12
FigureSize = (11, 8)

Schaal.sort()

# Functie om de om de rotatie van labels in de matrix te corrigeren gecentreerd
def SetLabelRotation(Axes):
    if not isinstance(Axes, np.ndarray):
        Axes = np.atleast_1d(Axes)

    if Axes.ndim == 2:
        for x in range(Axes.shape[0]):
            for y in range(Axes.shape[1]):
                plt.setp(Axes[x, y].get_xticklabels(), rotation=0)
                plt.setp(Axes[x, y].get_yticklabels(), rotation=0)
    else:
        for ax in np.atleast_1d(Axes):
            plt.setp(ax.get_xticklabels(), rotation=0)
            plt.setp(ax.get_yticklabels(), rotation=0)
    plt.tight_layout()

# Functie die de Labelmatch spelling correcties uitvoert en zorgt voor het opbreken van lange labels in meerdere lijnen met een max lengte van {MaxCharacterLength}
def CleanUpLabels(labels : list):
    NewLabels = []
    for label in labels:
        if label in LabelMatch.keys():
            label = LabelMatch[label]

        NewLabel = textwrap.fill(label, width= MaxCharacterLength, break_long_words= True)

        NewLabels.append(NewLabel)
    return NewLabels


# Negeer de specifieke waarschuwing die wordt gegenereerd door IPython
warnings.filterwarnings("ignore", message="To exit: use 'exit', 'quit', or Ctrl-D.", category=UserWarning)

# Check of opgegeven waarde bestaan
if TitleOverwrite and TitleOverwrite not in Clustering:
    sys.exit("De opgegeven 'TitleOverwrite' is niet teruggevonden in de opgegeven 'Clustering'. verander dit voor een succesvolle run")

OutputFolder.mkdir(parents=True, exist_ok=True)


# Leest het excel document in
InputExcel = pd.read_excel(InputExcelLocation, sheet_name="Matrix per data punt")

# Maakt een dictionary aan om bestand selectie aan toe te voegen. 
    # Keys = clusternamen
    # values = set van bestandsnamen. 
    #   De set() voorkomt fouten door bestanden met gelijke benamingen
FileSelection: dict[str, set[str]] = {}

TitleOverwriteList : dict[str, str]=  {}

# Sorteert de lijst met opgegeven clusters zodat figuren constant blijven
Clustering.sort()

# Loopt door opgegeven excel en sorteert alle samples in de correcte clusters
for Index, Row in InputExcel.iterrows():
    ClusterKey = []
    ValidClusterKey = True
    FirstLoop = True
    
    # Selecteert de correcte variabele per rij
    for Variabelen in Clustering:
        VariabelText = str(Row[f"Variabele {Variabelen}"])


        #Checkt of variable op blacklist staat
        if VariabelText in Blacklist:
            ValidClusterKey = False

        # Checkt en vervangt variabelen volgens de opgegeven VariabelenMatch
        for Key in VariabelenMatch:
            for String in VariabelenMatch[Key]:
                if String == VariabelText:
                    VariabelText = Key

        #Checkt of variable op blacklist staat
        if VariabelText in Blacklist:
            ValidClusterKey = False

        if TitleOverwrite:
            if TitleOverwrite == Variabelen:
                TitleOverwriteText = VariabelText
        
        ClusterKey.append(VariabelText)

    # merged de variabelen in de lijst tot een string welke als key in FileSelection gebruikt word en terug komt in de matrix titel
    ClusterKey = " - ".join(ClusterKey)
    
    # Checkt of de blacklist controle afgegaan is en voegt anders het sample to aan FileSelection
    if ValidClusterKey:
        if ClusterKey in FileSelection:
            FileSelection[ClusterKey].add(str(Row["SampleName"]))
        if not ClusterKey in FileSelection:
            FileSelection[ClusterKey] = {str(Row["SampleName"])}
            if TitleOverwrite:
                TitleOverwriteList[ClusterKey] = TitleOverwriteText

# maakt een lijst van columnnamen met 'Model: '. dit gaat er vanuit dat het excel afkomstig is van MassValidate.ipynb
 # deze data word gebruikt om de juiste data te selecteren en later om de labels voor de matrix te genereren
ColSelect = [col for col in InputExcel.columns if 'Model: ' in col]


for i, (Key, samples) in enumerate(FileSelection.items()):
    OutputMatrix = pd.DataFrame()

    # Loopt over alle matrixen heeb die opgeslagen zijn voor de datsets binnen het cluster en telt deze bij elkaar op tot een matrix per cluster/Key
    for DataSet in samples:
        DatasetMatrix = InputExcel.loc[InputExcel["SampleName"] == DataSet].loc[:, ColSelect].reset_index(drop=True)
        OutputMatrix = OutputMatrix.add(DatasetMatrix, fill_value=0)
    
    # Filtert de relevante Label namen uit de excel
    OutputMatrixLabels = [x.removeprefix("Model: ") for x in OutputMatrix.columns]
    OutputMatrixLabels = CleanUpLabels(OutputMatrixLabels)
    
    # Bereken de median van de hele matrix. word gebruikt om de schaalverdeling te bepalen
    Median = pd.DataFrame([x for rows in OutputMatrix.values.tolist() for x in rows])
    Median = Median[Median[0] != 0].median()
    
    # Alle 0 waardes worden omgezet naar NaN. Dit maakt de cellen in de output leeg anders weergeven ze een 0
    OutputMatrixNoZero = OutputMatrix.replace(0, np.nan).T

    # Genereer een losse matrix met de werkelijke waarde. 
    # vmin en vmax gebruiken een if statement om te kiezen tussen een User opgegeven schaal of de schaal op basis van de median
    fig_raw, ax_raw = plt.subplots(figsize=FigureSize)
    sns.set(font_scale=FontSize)
    sns.heatmap(
        OutputMatrixNoZero, 
        annot=True, 
        cmap=sns.color_palette("light:#77BFCB", as_cmap=True), 
        square=True, 
        xticklabels=OutputMatrixLabels, 
        yticklabels=OutputMatrixLabels,
        fmt=".0f",
        vmin=0 if not Schaal else Schaal[0],
        vmax=(Median * 2) if not Schaal else Schaal[1],
        ax=ax_raw,
        linewidth=LineWidth, 
        linecolor=LineColor,
    ).set_facecolor(FaceColor)

    SetLabelRotation(ax_raw)

    # Produceert de correcte titel op basis van input
    if TitleOverwrite:
        ax_raw.set_title(TitleOverwriteList[Key])
    elif Key:
        ax_raw.set_title(f"Confusion Matrix van {Key}")
    else: #wanneer er geen clusters gegeven worden
        ax_raw.set_title(f"Confusion Matrix")

    ax_raw.set_xlabel("Getelde individuen")
    ax_raw.set_ylabel("Voorspellingen model")
    
    # Slaat de losse matrix op onder een geschikte bestandsnaam
    if Key:
        OutputFileRaw = OutputFolder / f"{Key}_raw.png"
    else:
        OutputFileRaw = OutputFolder / f"Hele_dataset_raw.png"

    fig_raw.savefig(
        OutputFileRaw,
        dpi=Dpi,
    )
    plt.close(fig_raw)

    # Normalize the confusion matrix
    OutputMatrixNormalised = OutputMatrixNoZero.div(OutputMatrixNoZero.sum(axis=0), axis=1)

    # Genereer een losse matrix met de Genormaliseerde waarde. 
    # vmin en vmax zijn altijd 0 en 1 want er is genormaliseerd op schaal tussen 1 en 0
    fig_norm, ax_norm = plt.subplots(figsize=FigureSize)
    sns.set(font_scale=FontSize)
    sns.heatmap(
        OutputMatrixNormalised, 
        annot=True, 
        cmap=sns.color_palette("light:#88D27A", as_cmap=True), 
        square=True, 
        xticklabels=OutputMatrixLabels, 
        yticklabels=OutputMatrixLabels,
        fmt=".2f",
        vmin=0,
        vmax=1,
        ax=ax_norm,
        linewidth=LineWidth, 
        linecolor=LineColor,
    ).set_facecolor(FaceColor)

    SetLabelRotation(ax_norm)

    # Produceert de correcte titel op basis van input
    if TitleOverwrite:
        ax_norm.set_title(TitleOverwriteList[Key])
    elif Key:
        ax_norm.set_title(f"Normalized Confusion Matrix van {Key}")
    else: # wanneer er geen clusters gegeven worden
        ax_norm.set_title(f"Normalized Confusion Matrix")

    ax_norm.set_xlabel("Getelde individuen")
    ax_norm.set_ylabel("Voorspellingen model")

    # Slaat de losse matrix op onder een geschikte bestandsnaam    
    if Key:
        OutputFileNorm = OutputFolder / f"{Key}_normalized.png"
    else:
        OutputFileNorm = OutputFolder / f"Hele_dataset_normalized.png"
        
    fig_norm.savefig(
        OutputFileNorm,        
        dpi=Dpi,
    )
    plt.close(fig_norm)

# Generate overview grid om de samen gestelde matrixen in op te slaan
fig, axes = plt.subplots(len(FileSelection), 2, figsize=(FigureSize[0] * 2, FigureSize[1] * len(FileSelection)))
axes = np.atleast_2d(axes)

# Herhaal de loep om matrixen te maken maar nu in de overview grid
for i, (Key, samples) in enumerate(FileSelection.items()):
    OutputMatrix = pd.DataFrame()

    # Loopt over alle matrixen heeb die opgeslagen zijn voor de datsets binnen het cluster en telt deze bij elkaar op tot een matrix per cluster/Key
    for DataSet in samples:
        DatasetMatrix = InputExcel.loc[InputExcel["SampleName"] == DataSet].loc[:, ColSelect].reset_index(drop=True)
        OutputMatrix = OutputMatrix.add(DatasetMatrix, fill_value=0)
    
    # Filtert de relevante Label namen uit de excel
    OutputMatrixLabels = [x.removeprefix("Model: ") for x in OutputMatrix.columns]
    OutputMatrixLabels = CleanUpLabels(OutputMatrixLabels)
    
    # Bereken de median van de hele matrix. word gebruikt om de schaalverdeling te bepalen
    Median = pd.DataFrame([x for rows in OutputMatrix.values.tolist() for x in rows])
    Median = Median[Median[0] != 0].median()
    
    # Alle 0 waardes worden omgezet naar NaN. Dit maakt de cellen in de output leeg anders weergeven ze een 0
    OutputMatrixNoZero = OutputMatrix.replace(0, np.nan).T

    # Genereer een losse matrix in het grid met de werkelijke waarde. 
    # vmin en vmax gebruiken een if statement om te kiezen tussen een User opgegeven schaal of de schaal op basis van de median
    sns.set(font_scale=FontSize)
    sns.heatmap(
        OutputMatrixNoZero, 
        annot=True, 
        cmap=sns.color_palette("light:#77BFCB", as_cmap=True), 
        square=True, 
        xticklabels=OutputMatrixLabels, 
        yticklabels=OutputMatrixLabels,
        fmt=".0f",
        vmin=0 if not Schaal else Schaal[0],
        vmax=(Median * 2) if not Schaal else Schaal[1],
        ax=axes[i, 0],
        linewidth=LineWidth, 
        linecolor=LineColor,
    ).set_facecolor(FaceColor)

    # Produceert de correcte titel op basis van input
    if TitleOverwrite:
        axes[i, 0].set_title(TitleOverwriteList[Key])
    elif Key:
        axes[i, 0].set_title(f"Confusion Matrix van {Key}")
    else: #wanneer er geen clusters gegeven worden
        axes[i, 0].set_title(f"Confusion Matrix")

    axes[i, 0].set_xlabel("Getelde individuen")
    axes[i, 0].set_ylabel("Voorspellingen model")

    # Normalize the confusion matrix
    OutputMatrixNormalised = OutputMatrixNoZero.div(OutputMatrixNoZero.sum(axis=0), axis=1)

    # Genereer een losse matrix in het grid met de Genormaliseerde waarde. 
    # vmin en vmax zijn altijd 0 en 1 want er is genormaliseerd op schaal tussen 1 en 0
    sns.heatmap(
        OutputMatrixNormalised, 
        annot=True, 
        cmap=sns.color_palette("light:#88D27A", as_cmap=True), 
        square=True, 
        xticklabels=OutputMatrixLabels, 
        yticklabels=OutputMatrixLabels,
        fmt=".2f",
        vmin=0,
        vmax=1,
        ax=axes[i, 1],
        linewidth=LineWidth, 
        linecolor=LineColor,
    ).set_facecolor(FaceColor)

    SetLabelRotation(axes)

    # Produceert de correcte titel op basis van input
    if TitleOverwrite:
        axes[i, 1].set_title(TitleOverwriteList[Key])
    elif Key:
        axes[i, 1].set_title(f"Normalized Confusion Matrix van {Key}")
    else: # wanneer er geen clusters gegeven worden        
        axes[i, 1].set_title(f"Normalized Confusion Matrix")
    
    axes[i, 1].set_xlabel("Getelde individuen")
    axes[i, 1].set_ylabel("Voorspellingen model")

# Slaat de overview matrix op onder een geschikte bestandsnaam   
if Clustering:
    OverviewFile = OutputFolder / f"overview {Clustering}.png"
else:
    OverviewFile = OutputFolder / f"overview hele dataset.png"

fig.savefig(
    OverviewFile,
    dpi=Dpi,
    )